# Practice Session 04: Basket analysis

Author: <font color="white">Mario Muñoz Serrano</font>

Date: <font color="white">21/10/2022</font>

In [ ]:
!pip install apyori

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np  
import matplotlib.pyplot as plt  
import pandas as pd  
import csv
import gzip
from apyori import apriori

# 1. Playing with apyori

In [ ]:
# LEAVE AS-IS

def print_apyori_output (association_results, info=False, info_key=False):
    for relation_record in association_results:
        itemset = list(relation_record.items)
        
        # Consider only itemsets of two elements
        if len(itemset) > 1: 
        
            print("Rules involving itemset %s" % itemset)
            support = relation_record.support

            for rules in relation_record.ordered_statistics:
                antecedent = list(rules.items_base)
                consequent = list(rules.items_add)
                
                if info_key:
                    antecedent = [info.loc[x][info_key] for x in antecedent]
                    consequent = [info.loc[x][info_key] for x in consequent]
                
                confidence = rules.confidence
                lift = rules.lift

                print("%s => %s (support=%.4f, confidence=%.2f, lift=%.2f)" %
                      (antecedent, consequent, support, confidence, lift))
            print()

In [ ]:
def get_transactions(url):
    clean = pd.read_csv(url, sep=';')

    clean.drop(clean.tail(1).index,inplace=True)


    transactions = []

    for index in clean.index:
        row = clean.iloc[index,:]
        private_num = row['Privat o concertat']
        public_num = row['Public']

        for student in range(private_num):
            transaction = []
            transaction.extend((row['Universitat'], row['Grau'], 'Private_School'))
            


            transactions.append(transaction)


        for student in range(public_num):
            transaction = []
            transaction.extend((row['Universitat'], row['Grau'], 'Public_School'))
            


            transactions.append(transaction)
            
    return transactions

In [ ]:
# Data obtained from
# La segregació invisible de les universitats. (s/f). Crític. Retrived 21 of octubre of 2022, 
# from https://www.elcritic.cat/dades/la-segregacio-invisible-de-les-universitats-112209


transactions = get_transactions('https://raw.githubusercontent.com/mariomunooz/data/main/KW9FwtY9.csv')

In [ ]:
results = list(apriori(transactions, min_support=(800/len(transactions)), min_confidence=0.75, min_lift=20.0))

print_apyori_output(results)

Rules involving itemset ['Criminologia i Polítiques Públiques de Prevenció', 'Public_School', 'UPF']
['Criminologia i Polítiques Públiques de Prevenció'] => ['Public_School', 'UPF'] (support=0.0019, confidence=0.89, lift=23.55)

Rules involving itemset ['Enginyeria Matemàtica en Ciència de Dades / Mathematical Engineering on Data Science', 'Public_School', 'UPF']
['Enginyeria Matemàtica en Ciència de Dades / Mathematical Engineering on Data Science'] => ['Public_School', 'UPF'] (support=0.0016, confidence=0.98, lift=26.00)

Rules involving itemset ['UPF', 'Private_School', 'Estudis Globals / Global Studies']
['Estudis Globals / Global Studies'] => ['UPF', 'Private_School'] (support=0.0016, confidence=0.98, lift=20.46)



In [ ]:
def compute_suport(transactions, A, print_flag):
  transactions_containing_A = 0
  
  for transaction in transactions:

    if all(x in transaction for x in A):
      transactions_containing_A += 1

  support = round(transactions_containing_A/len(transactions), 4)

  if print_flag:
    print(f'Support({A}) = (Transactions containing ({A}))/(Total Transactions)')
    print(f'Support({A}) = ({transactions_containing_A})/({len(transactions)}) = {support} \n')

  return support

In [ ]:
def compute_confidence(transactions, A, B, print_flag):

  transactions_containing_A_and_B = 0
  transactions_containing_A = 0



  for transaction in transactions:
    if all(x in transaction for x in A) and all(y in transaction for y in B):
        transactions_containing_A_and_B += 1
    
    if all(x in transaction for x in A):
      transactions_containing_A += 1

  confidence = round(transactions_containing_A_and_B/transactions_containing_A, 2)

  if print_flag:
    print(f'Confidence({A}-> {B}) = (Transactions containing ({A} and {B}))/Transactions containing ({A})')
    print(f'Confidence({A}-> {B}) = ({transactions_containing_A_and_B})/({transactions_containing_A}) = {confidence} \n')

  return confidence


In [ ]:
def computations(transactions, A, B):
  print('SUPPORT')
  supportA = compute_suport(transactions, A, True)
  supportB = compute_suport(transactions, B, False)
  print('CONFIDENCE')
  confidence = compute_confidence(transactions, A, B, True)
  print('LIFT')


  print(f'Lift({A}->{B}) = (Confidence ({A}->{B}))/(Support ({B}))')
  print(f'Lift({A}->{B}) = ( ({round(confidence/ supportB, 4)})\n')

In [ ]:
computations(transactions, ['Enginyeria Matemàtica en Ciència de Dades / Mathematical Engineering on Data Science'], ['Public_School', 'UPF'])
computations(transactions, ['Estudis Globals / Global Studies'], ['UPF', 'Private_School'])
computations(transactions, ['Criminologia i Polítiques Públiques de Prevenció'], ['Public_School', 'UPF'])

SUPPORT
Support(['Enginyeria Matemàtica en Ciència de Dades / Mathematical Engineering on Data Science']) = (Transactions containing (['Enginyeria Matemàtica en Ciència de Dades / Mathematical Engineering on Data Science']))/(Total Transactions)
Support(['Enginyeria Matemàtica en Ciència de Dades / Mathematical Engineering on Data Science']) = (3774)/(2309420) = 0.0016 

CONFIDENCE
Confidence(['Enginyeria Matemàtica en Ciència de Dades / Mathematical Engineering on Data Science']-> ['Public_School', 'UPF']) = (Transactions containing (['Enginyeria Matemàtica en Ciència de Dades / Mathematical Engineering on Data Science'] and ['Public_School', 'UPF']))/Transactions containing (['Enginyeria Matemàtica en Ciència de Dades / Mathematical Engineering on Data Science'])
Confidence(['Enginyeria Matemàtica en Ciència de Dades / Mathematical Engineering on Data Science']-> ['Public_School', 'UPF']) = (3714)/(3774) = 0.98 

LIFT
Lift(['Enginyeria Matemàtica en Ciència de Dades / Mathematical En

# 2. Load and prepare the shopping baskets

In [ ]:
# LEAVE AS-IS

# File names
# Products data in https://github.com/chatox/data-mining-course/tree/master/practicum/data/instacart
INPUT_PRODUCTS = "instacart-products.csv"
INPUT_TRANSACTIONS = "instacart-transactions.csv.gz"

# Read into a dataframe
products = pd.read_csv(INPUT_PRODUCTS, delimiter=",")

# Set product_id as index, and drop column aisle_id
products = products.set_index('product_id').drop(columns=['aisle_id'])

products.head(100)

,product_name,department_id
product_id,,
1,Chocolate Sandwich Cookies,19
2,All-Seasons Salt,13
3,Robust Golden Unsweetened Oolong Tea,7
4,Smart Ones Classic Favorites Mini Rigatoni Wit...,1
5,Green Chile Anytime Sauce,13
...,...,...
96,Sprinklez Confetti Fun Organic Toppings,13
97,Organic Chamomile Lemon Tea,7
98,2% Yellow American Cheese,16


## 2.1. Select by department

In [ ]:
# LEAVE AS-IS

DEPT_BAKERY = 3
DEPT_VEGGIES = 4
DEPT_ALCOHOL = 5
DEPT_WORLD = 6
DEPT_DRINKS = 7
DEPT_PETS = 8
DEPT_PHARMACY = 11
DEPT_CLEANING = 17
DEPT_BABIES = 18

In [ ]:
# select_from_departments that takes as input:
# 
#     A dataframe containing product information, which will be the products dataframe we just loaded.
#     A list of product ids
#     A list of department ids
# 
# It should return a list containing only the product ids that belong to one of the listed departments. 
# This may return an empty list if no product belongs to any of the specified departments.

def select_from_departments(products_df, product_ids_list, deparment_ids_list, print_flag):
  input_products = products_df.loc[product_ids_list]
  selected_products = input_products[input_products.department_id.isin(deparment_ids_list)]

  selected_products_list = selected_products.index.values.tolist()

  if (print_flag):
    print(f'Test case:\n{product_ids_list}\n\n\nInput products:')

    for i in input_products.index.values.tolist():
      row = input_products.loc[i]
      print(f'{i} {row["product_name"]} (dept {row["department_id"]})')


    print(f'\n\nSelcted products:\n\n')

    for i in selected_products.index.values.tolist():
      row = selected_products.loc[i]
      print(f'{i} {row["product_name"]} (dept {row["department_id"]})')


  return selected_products_list


In [ ]:
b = select_from_departments(products, [21, 26, 45, 54, 57, 71, 111, 112], [8,17], True)

Test case:
[21, 26, 45, 54, 57, 71, 111, 112]


Input products:
21 Small & Medium Dental Dog Treats (dept 8)
26 Fancy Feast Trout Feast Flaked Wet Cat Food (dept 8)
45 European Cucumber (dept 4)
54 24/7 Performance Cat Litter (dept 8)
57 Flat Toothpicks (dept 17)
71 Ultra 7 Inch Polypropylene Traditional Plates (dept 17)
111 Fabric Softener, Geranium Scent (dept 17)
112 Hot Tomatillo Salsa (dept 13)


Selcted products:


21 Small & Medium Dental Dog Treats (dept 8)
26 Fancy Feast Trout Feast Flaked Wet Cat Food (dept 8)
54 24/7 Performance Cat Litter (dept 8)
57 Flat Toothpicks (dept 17)
71 Ultra 7 Inch Polypropylene Traditional Plates (dept 17)
111 Fabric Softener, Geranium Scent (dept 17)


## 2.2. Read and filter transactions

In [ ]:
from IPython.core.interactiveshell import traceback
th = 5000

transactions = []
c = 0
# Open a compressed file
with gzip.open(INPUT_TRANSACTIONS, "rt") as inputfile:

    # Create a CSV reader
    reader = csv.reader(inputfile, delimiter=",")

    # Iterate through the CSV file
    for row in reader:

        # Convert to integers
        items = [int(x) for x in row]

        filtered_t = select_from_departments(products, items, [DEPT_CLEANING], False)
        if filtered_t:
          transactions.append(filtered_t)
          

        
        c += 1
        if(c % 1000 == 0): print(f'{c} transactions readed {len(transactions)} of transactions stored')

        if len(transactions) > th:
          break


1000 transactions readed 158 of transactions stored
2000 transactions readed 311 of transactions stored
3000 transactions readed 460 of transactions stored
4000 transactions readed 598 of transactions stored
5000 transactions readed 745 of transactions stored
6000 transactions readed 902 of transactions stored
7000 transactions readed 1067 of transactions stored
8000 transactions readed 1206 of transactions stored
9000 transactions readed 1373 of transactions stored
10000 transactions readed 1515 of transactions stored
11000 transactions readed 1670 of transactions stored
12000 transactions readed 1807 of transactions stored
13000 transactions readed 1951 of transactions stored
14000 transactions readed 2102 of transactions stored
15000 transactions readed 2245 of transactions stored
16000 transactions readed 2384 of transactions stored
17000 transactions readed 2543 of transactions stored
18000 transactions readed 2692 of transactions stored
19000 transactions readed 2840 of transacti

## 2.3. Extract association rules and comment on them (DEPT_CLEANING)

In [ ]:
total_number_of_transactions = th
results = list(apriori(transactions, min_support= 3/total_number_of_transactions, min_confidence=0.5, min_lift=10.0))
print_apyori_output(results, products, 'product_name')

Rules involving itemset [15341, 717]
['Sweeper Dry Sweeping Refills'] => ['Sweeper Open Window Fresh Scent Wet Mopping Cloths Refill'] (support=0.0008, confidence=0.50, lift=192.35)

Rules involving itemset [37835, 13516]
['Chlorine Free Versatile Stain Remover'] => ['Laundry Stain Remover Spray'] (support=0.0008, confidence=0.57, lift=317.52)

Rules involving itemset [18229, 21653]
['Plastic Knives'] => ['Compostable Forks'] (support=0.0008, confidence=1.00, lift=250.05)



I would make the following recomendations to the users, if someone buys:

 

*   Sweeper Dry Sweeping Refills. Do you want also: Sweeper Open Window Fresh Scent Wet Mopping Cloths Refill?
*   Chlorine Free Versatile Stain Remover. Do you want also: Laundry Stain Remover Spray?
*   Plastic Knives. Do you want also: Compostable Forks?



## 2.4. Extract association rules and comment on them (other departments)

In [ ]:
transactions = []
c = 0
# Open a compressed file
with gzip.open(INPUT_TRANSACTIONS, "rt") as inputfile:

    # Create a CSV reader
    reader = csv.reader(inputfile, delimiter=",")

    # Iterate through the CSV file
    for row in reader:

        # Convert to integers
        items = [int(x) for x in row]

        filtered_t = select_from_departments(products, items, [DEPT_PHARMACY, DEPT_BABIES], False)
        if filtered_t:
          transactions.append(filtered_t)
          

        
        c += 1
        if(c % 1000 == 0): print(f'{c} transactions readed {len(transactions)} of transactions stored')

        if len(transactions) > th:
          break

1000 transactions readed 149 of transactions stored
2000 transactions readed 316 of transactions stored
3000 transactions readed 467 of transactions stored
4000 transactions readed 619 of transactions stored
5000 transactions readed 774 of transactions stored
6000 transactions readed 911 of transactions stored
7000 transactions readed 1065 of transactions stored
8000 transactions readed 1195 of transactions stored
9000 transactions readed 1348 of transactions stored
10000 transactions readed 1530 of transactions stored
11000 transactions readed 1688 of transactions stored
12000 transactions readed 1824 of transactions stored
13000 transactions readed 1983 of transactions stored
14000 transactions readed 2149 of transactions stored
15000 transactions readed 2306 of transactions stored
16000 transactions readed 2457 of transactions stored
17000 transactions readed 2588 of transactions stored
18000 transactions readed 2734 of transactions stored
19000 transactions readed 2862 of transacti

In [ ]:
results = list(apriori(transactions, min_support= 10/total_number_of_transactions, min_confidence=0.6, min_lift=10.0))
print_apyori_output(results, products, 'product_name')

Rules involving itemset [32018, 38141]
['Fiber & Protein Organic Pears, Raspberries, Butternut Squash & Carrots Snack'] => ['Organic Fiber & Protein Pear Blueberry & Spinach Baby Food'] (support=0.0034, confidence=0.77, lift=107.34)

Rules involving itemset [32018, 45495]
['Pear Kiwi & Kale Baby Food'] => ['Organic Fiber & Protein Pear Blueberry & Spinach Baby Food'] (support=0.0034, confidence=0.71, lift=98.40)

Rules involving itemset [32018, 38141, 45495]
['Organic Fiber & Protein Pear Blueberry & Spinach Baby Food', 'Fiber & Protein Organic Pears, Raspberries, Butternut Squash & Carrots Snack'] => ['Pear Kiwi & Kale Baby Food'] (support=0.0022, confidence=0.65, lift=134.83)
['Organic Fiber & Protein Pear Blueberry & Spinach Baby Food', 'Pear Kiwi & Kale Baby Food'] => ['Fiber & Protein Organic Pears, Raspberries, Butternut Squash & Carrots Snack'] (support=0.0022, confidence=0.65, lift=147.09)
['Fiber & Protein Organic Pears, Raspberries, Butternut Squash & Carrots Snack', 'Pear Ki

I would make the following recomendations to the users, if someone buys:
*    ['Fiber & Protein Organic Pears, Raspberries, Butternut Squash & Carrots Snack']. Do you want also: ['Organic Fiber & Protein Pear Blueberry & Spinach Baby Food']
*    ['Pear Kiwi & Kale Baby Food']. Do you want also: ['Organic Fiber & Protein Pear Blueberry & Spinach Baby Food']
*    ['Organic Fiber & Protein Pear Blueberry & Spinach Baby Food', 'Fiber & Protein Organic Pears, Raspberries, Butternut Squash & Carrots Snack']. Do you want also: ['Pear Kiwi & Kale Baby Food']
*    ['Organic Fiber & Protein Pear Blueberry & Spinach Baby Food', 'Pear Kiwi & Kale Baby Food']. Do you want also: ['Fiber & Protein Organic Pears, Raspberries, Butternut Squash & Carrots Snack']